In [ ]:
import pandas as pd
import requests
import hashlib

In [3]:
df = pd.read_parquet("logos.snappy.parquet")

In [ ]:
def fetch_logo(domain, timeout=5):
    url_clearbit = f"https://logo.clearbit.com/{domain}"
    try:
        response = requests.get(url_clearbit, timeout=timeout)
        if response.status_code == 200 and response.content:
            return response.content
    except Exception as e:
        pass
    
    "Foloseste API-ul Google Favicons"
    url_google = f"https://www.google.com/s2/favicons?sz=64&domain_url={domain}"
    try:
        response = requests.get(url_google, timeout=timeout)
        if response.status_code == 200 and response.content:
            return response.content
    except Exception as e:
        pass
    
    return None

def get_logo_hash(logo_bytes):
    if logo_bytes:
        return hashlib.md5(logo_bytes).hexdigest()
    return None

df['logo_bytes'] = df['domain'].apply(fetch_logo)
df['logo_hash'] = df['logo_bytes'].apply(get_logo_hash)
total = len(df)
valid_logos = df['logo_hash'].notna().sum()
percent_with_logo = valid_logos / total * 100
print(f"Percentage of websites with logo: {percent_with_logo:.2f}%")
if percent_with_logo < 97:
    print("Warning: Logos extracted for less than 97% of websites.")
else:
    print("Success: Logos extracted for more than 97% of websites.")

groups = df[df['logo_hash'].notna()].groupby('logo_hash')['domain'].apply(list).reset_index()
groups.to_csv("grouped_logos.csv", index=False)
groups.to_parquet("grouped_logos.snappy.parquet", compression="snappy")

Percentage of websites with logo: 97.65%
Success: Logos extracted for more than 97% of websites.


In [25]:
def fetch_logo(domain, timeout=5):
    "Foloseste API-ul Google Favicons"
    url_google = f"https://www.google.com/s2/favicons?sz=64&domain_url={domain}"
    try:
        response = requests.get(url_google, timeout=timeout)
        if response.status_code == 200 and response.content:
            return response.content
    except Exception as e:
        pass
    
    return None

def get_logo_hash(logo_bytes):
    if logo_bytes:
        return hashlib.md5(logo_bytes).hexdigest()
    return None

df['logo_bytes'] = df['domain'].apply(fetch_logo)
df['logo_hash'] = df['logo_bytes'].apply(get_logo_hash)
total = len(df)
valid_logos = df['logo_hash'].notna().sum()
percent_with_logo = valid_logos / total * 100
print(f"Percentage of websites with logo: {percent_with_logo:.2f}%")
if percent_with_logo < 97:
    print("Warning: Logos extracted for less than 97% of websites.")
else:
    print("Success: Logos extracted for more than 97% of websites.")

groups = df[df['logo_hash'].notna()].groupby('logo_hash')['domain'].apply(list).reset_index()
groups.to_csv("grouped_logos.csv", index=False)
groups.to_parquet("grouped_logos.snappy.parquet", compression="snappy")

Percentage of websites with logo: 92.34%
